In [2]:


import xlrd
import pymysql
import math
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup 
import requests 
import re 
import random


In [198]:
conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()
cur.execute("SET NAMES utf8")

sql_cmd = "SELECT * FROM music.Excel13;"
df = pd.read_sql(sql_cmd,conn)
df





/usr/local/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (3719, "'utf8' is currently an alias for the character set UTF8MB3, but will be an alias for UTF8MB4 in a future release. Please consider using UTF8MB4 in order to be unambiguous.")
  result = self._query(query)


,id,link,class,singer_id,singer_name,song_id,song_name,song_word,album_id,album_name,album_language,album_song,years,month,lyric_id,same_id,lyric_language,composer_name,author_name
0,100000,https://mojim.com/twy100198x2x2.htm,男生歌手,100198,張洪量,1001980202,情為何物,4,10019802,整個給你,國語,10,1993,1,100000,100000,Chinese,張洪量,張洪量
1,100002,https://mojim.com/twy100198x2x4.htm,男生歌手,100198,張洪量,1001980204,浮萍,2,10019802,整個給你,國語,10,1993,1,100002,100002,Chinese,張洪量,張洪量
2,100005,https://mojim.com/twy100198x2x7.htm,男生歌手,100198,張洪量,1001980207,沙漠之花,4,10019802,整個給你,國語,10,1993,1,100005,100005,Chinese,張洪量,張洪量
3,100006,https://mojim.com/twy100198x2x8.htm,男生歌手,100198,張洪量,1001980208,我想我瘋了,5,10019802,整個給你,國語,10,1993,1,100006,100006,Chinese,張洪量,張洪量
4,100007,https://mojim.com/twy100198x2x9.htm,男生歌手,100198,張洪量,1001980209,外星戀,3,10019802,整個給你,國語,10,1993,1,100007,100007,Chinese,張洪量,張洪量
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
261484,293084,https://mojim.com/twy105001x1x1.htm,其他歌手,105001,網友音樂創作,1050010101,痛徹心扉(孫培恩),4,10500101,痛徹心扉,國語,1,未知,未知,293084,293084,Chinese,未知,未知
261485,293085,https://mojim.com/twy105001x2x1.htm,其他歌手,105001,網友音樂創作,1050010201,朝代,2,10500102,朝代,國語,1,未知,未知,293085,293085,Chinese,未知,口香糖
261486,293086,https://mojim.com/twy105001x3x1.htm,其他歌手,105001,網友音樂創作,1050010301,愛情的樂章,5,10500103,愛情的樂章,國語,1,未知,未知,293086,293086,Chinese,未知,未知
261487,293087,https://mojim.com/twy105001x4x1.htm,其他歌手,105001,網友音樂創作,1050010401,妳離開的事實,6,10500104,你離開的事實,國語,1,未知,未知,293087,293087,Chinese,PianoBoy(The truth that you leave),小凱


In [204]:
df1=df[['singer_id']].drop_duplicates()
df1 

,singer_id
0,100198
145,100146
544,100431
662,100362
755,122502
...,...
261475,160726
261477,169267
261478,186221
261480,134729


In [222]:
df2=df[['singer_id','singer_name']].drop_duplicates()
df2.head(1)

,singer_id,singer_name
0,100198,張洪量


In [206]:
# df1[df1['singer_id'].str.contains('twy')]

In [207]:
sin_id=df1['singer_id'].values.tolist()
print(len(album_id))
sin_id[:5]

8240


[100198, 100146, 100431, 100362, 122502]

In [224]:
list2=sin_id[6135:6137]
df2[df2['singer_id'].isin(list2)]

,singer_id,singer_name
255630,161716,林菁茹
255631,121604,及時雨


In [229]:
conn = pymysql.connect(
        host='localhost', 
        user='root', 
        passwd = 'fc117108',
        db='music',
        port=3306,
        charset='utf8'
        )
cur = conn.cursor()
def get(i,no):
    
    start=0
    tag=''
    
#     print(i,type(i))
    url='https://mojim.com/twh'+i+'.htm'
    singer_link=i
#     print(url)
    resp = requests.get(url) #  
    soup = BeautifulSoup(resp.text, 'html.parser')
    singer=soup.find('a',href=url).string
#     print(singer)
    a=soup.find('dl',class_='ha0').find_all('dd')
    values=[]
    for inx1,aa in enumerate(a[:]):
#         print(inx1,'aa',aa.text,start)
        if ('年份')in aa.text and inx1>1 and start==0 :
            tag='O'
            start=inx1
#             print('年份')
        elif ('日期') in aa.text and inx1>1 and start==0 :
            tag='X'
            start=inx1
#             print('日期')

            
#         print('inx1',inx1,'start',start,'tag',tag)
        
        if inx1 >start:
            if tag=='O':
                album_name=aa.find(class_='hc1').text
                aaa=aa.find(class_='hc2')
                data=[]
                for l in aaa:
                    if l.string is not None:
                        data.append(l.string)
                data1=data
#                 print('data1',data1)
                for i in data:
                    if  bool(re.search(r'\d', i)):
                        year=i
                        data1.remove(year)
                    else:
                        year='未記錄'
                    lan=data1[0] if len(data1)>1 else '未記錄'
                    com=data1[1] if len(data1)>1 else '未記錄'

                
                song1=aa.find(class_='hc3').find_all('a')
                
                for s1 in song1:
                    song=s1['title'].split('歌詞')[0]
                    ly_url='https://mojim.com'+s1['href']

                    if 'thx' not in ly_url :
                        no+=1
                        values.append((no,singer_link,singer,album_name,song,ly_url,year,lan,com))
                
                song2=aa.find(class_='hc4').find_all('a')
                for s2 in song2:
                    song=s2['title'].split('歌詞')[0]            
                    ly_url='https://mojim.com/'+s2['href']
                    if 'thx' not in ly_url : 
                        no+=1
                        values.append((no,singer_link,singer,album_name,song,ly_url,year,lan,com))
                
            elif tag=='X':
                song3=aa.find(class_='hc1')
#                 print(song3)
                if song3 is not None:
                    for j in song3.find_all('a'):
                        no+=1
                        album_name=j.string
                        ly_url='https://mojim.com'+j['href']
                        year=''
                        lan=''
                        com=''
                        song=j.string
                        values.append((no,singer_link,singer,album_name,song,ly_url,year,lan,com))

#     print(no,'values',len(values),values,'\n')
    sql = "INSERT IGNORE INTO new (no,singer_link,singer,album,song,lyurl,year,langrage,compancy) VALUES (%s,%s, %s, %s, %s, %s, %s, %s, %s)"                      
    cur.executemany(sql, tuple(values))    
    conn.commit()
    return no

# list1=['100362']
no=295000
# for i in list1[:]:
for i in sin_id[6136:]:
    no=get(str(i),no)
print('finish')
conn.close()

finish
